In [98]:
import sqlite3
import json
from datetime import datetime
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.layers import Embedding
from keras.layers import Flatten
from keras.preprocessing.text import Tokenizer
from keras.datasets import imdb
from keras import preprocessing
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import os
from spacy.lang.en import English
from keras.layers import SimpleRNN
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras import utils
import re
import itertools
from sklearn.preprocessing import OneHotEncoder

# Importing data from a medical Q&A dataset containing over 135,000 questions and answer pairs

In [99]:
with open("healthtapQAs.json") as f: # This is the biggest database - 137,052 questions and answers
    my_data1 = json.load(f)

In [100]:
the_answers = list()
the_questions = list()

count = 0
count1 = 0

for i in my_data1:
    the_answers.append(i['answer'])
    count += 1

for i in my_data1:
    the_questions.append(i['question'])
    count1 += 1
    
print(the_questions[0])
print(the_answers[0])

zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.
a majority of the dental implants placed are titanium. they are highly successful with many years use ; many studies much lower in cost ; have many restorative options. zirconia implants are newer fewer studies on success lesser restorative options. however they can be more aesthetic in certain anterior(front) situations. let your dentist/oral surgeon chose what they feel will be best for you.


# Cleaning the data to get rid of some contractions 
## In the future I am thinking about applying this to each question written by a user

In [101]:
def clean_text(text):

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [102]:
answers = list()
questions = list()
for i in the_answers:
    text = clean_text(i)
    answers.append(text)

for i in the_questions:
    text = clean_text(i)
    questions.append(text)
    
print(the_questions[:4])
print(questions[:4])

['zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.', 'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.', 'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.', 'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.']
['zirconium dental implants how common is it used now is there any advantages or benefits over titanium implants cons & pros please thanks', 'zirconium dental implants how common is it used now is there any advantages or benefits over titanium implants cons & pros please thanks', 'zirconium dental implants how common is it used now is there any advantages or benefits over titanium implants cons & pros please thanks'

# Adding 'START' and 'END' to the beginnings and endings of the answers

In [103]:
print(answers[1:4])
labeled_answers = list()
for i in range(len(answers)):
    labeled_answers.append('<START>' + answers[i] + '<END>')


['and the data on zirconia implants is much more limited', 'dental implants when loaded transfer stress to the bone which is a good thing zirconium is a ceramic that is very hard and actually transfers less stress to the bone and can actually lead to bone atrophy  the you dont use it you lose it concept', 'stick with what we know works  titanium let the research continue with the zirconium more answers will come as more research is done hard to beat 50 years of success always looking for improvement though']


# Tokenizing words in both the questions and answers list

In [104]:
max_words = 20000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(questions + labeled_answers)
print("Vocab list has: {} elements".format(len(tokenizer.word_index)))
word_index = tokenizer.word_index

Vocab list has: 87816 elements


# Preprocessing Questions Data
## - Tokenize words
## - Create word integer index
## - Turn text to number sequence and padding

In [105]:
sequences_questions = tokenizer.texts_to_sequences(questions)
max_seq_len = max([len(x) for x in sequences_questions])
print("Longest sequence_questions is: {}".format(max_seq_len))
padded_questions = pad_sequences(sequences_questions, maxlen=max_seq_len)
# array of padded sequences for questions
encoder_input_data = np.array(padded_questions) 

print("Sequences are of dimensions: {}".format(encoder_input_data.shape))
print("Example sequence: {}".format(encoder_input_data[0]))
print("Example of WordIndex: {}".format(dict(itertools.islice(word_index.items(), 4))))

Longest sequence_questions is: 64
Sequences are of dimensions: (137052, 64)
Example sequence: [    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0 15221  1308   201    45   140     4
    11   155   136     4    38    65  3576    21   830   128  8161   201
  2943  3163   149  1030]
Example of WordIndex: {'the': 1, 'to': 2, 'a': 3, 'is': 4}


# Preprocessing Answers Data

In [106]:
sequences_answers = tokenizer.texts_to_sequences(labeled_answers)
max_seq_lenAns = 100
# max_seq_lenAns = max([len(x) for x in sequences_answers])
print("Longest sequence_answers is: {}".format(max_seq_lenAns))
padded_answers = pad_sequences(sequences_answers, maxlen=max_seq_lenAns)
# array of padded sequences for questions
decoder_input_data = np.array(padded_answers) 

print("Sequences are of dimensions: {}".format(decoder_input_data.shape))
print("Example sequence: {}".format(encoder_input_data[0]))
print("Example of WordIndex: {}".format(dict(itertools.islice(word_index.items(), 4))))

Longest sequence_answers is: 100
Sequences are of dimensions: (137052, 100)
Example sequence: [    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0 15221  1308   201    45   140     4
    11   155   136     4    38    65  3576    21   830   128  8161   201
  2943  3163   149  1030]
Example of WordIndex: {'the': 1, 'to': 2, 'a': 3, 'is': 4}


# Decoder output data

In [107]:
#experiment with the output dim 
output_dim = 200
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(max_words, output_dim, mask_zero=True) (encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(output_dim, return_state=True) (encoder_embedding)
# encoder_states = state_h, state_c


decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(max_words, output_dim, mask_zero=True) (decoder_inputs)
decoder_LSTM = LSTM(output_dim, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = Dense(max_words, activation='softmax')
output = decoder_dense (decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], output)

In [108]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [109]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, None, 200)    4000000     input_13[0][0]                   
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, None, 200)    4000000     input_14[0][0]                   
____________________________________________________________________________________________

# Removing the word START from the answer sequences and padding the sequences

In [110]:
decoder_output_seq = sequences_answers
for i in range(len(decoder_output_seq)):
    decoder_output_seq[i] = decoder_output_seq[i][1:]

target_data = pad_sequences(decoder_output_seq, max_seq_lenAns)


# Shuffling then splitting the input data and target data 80:20 

In [111]:
# should all share the shape (137052, _)
print("Encoder Input: ", encoder_input_data.shape)
print("Decoder Input: ", decoder_input_data.shape)
print("Target Data Input: ", target_data.shape)

Encoder Input:  (137052, 64)
Decoder Input:  (137052, 100)
Target Data Input:  (137052, 100)


In [112]:
eighty = 0.8 * encoder_input_data.shape[0]
print(eighty)
train_perc = 109500/encoder_input_data.shape[0]
print(train_perc)

109641.6
0.7989668155152789


In [113]:
# Since I will be fitting the model in portion sizes of 300 I am going
# use 109,500 which is approximately 80% (actually 79.897%)

In [114]:
indices = np.arange(encoder_input_data.shape[0])
np.random.shuffle(indices) 
# EID = encoder_input_data
shuffled_EID = encoder_input_data[indices]
shuffled_DID = decoder_input_data[indices]
shuffled_target = target_data[indices]

print(shuffled_EID.shape)
print(shuffled_DID.shape)
print(shuffled_target.shape)

(137052, 64)
(137052, 100)
(137052, 100)


In [115]:
train_encoder_data = shuffled_EID[0:109500] 
train_decoder_data = shuffled_DID[0:109500]
train_target_data = shuffled_target[0:109500]

test_encoder_data = shuffled_EID[109500: ] 
test_decoder_data = shuffled_DID[109500: ]
test_target_data = shuffled_target[109500: ]

print(train_encoder_data.shape)
print(train_decoder_data.shape)
print(train_target_data.shape)
print(test_encoder_data.shape)
print(test_decoder_data.shape)
print(test_target_data.shape)

print(train_encoder_data[1])
print(train_decoder_data[1])
print(train_target_data[1])

(109500, 64)
(109500, 100)
(109500, 100)
(27552, 64)
(27552, 100)
(27552, 100)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0    10    75   977   641   185   172  2815
    75 13875    48  4032  5894   151   227  4074     4    11   433   727
    10    71    42   379]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   7 285  19   8]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0  

# Fitting the model in portions since the OneHotEncoded Answers are too large to be held in computer memory

In [116]:
tenth = 300
times_completed = int(train_target_data.shape[0]/300)
for i in range(5):
    current = i * tenth
    nextone = (i+1) * tenth
    print("Currently--> {}:{}".format(current, nextone))
    train_X1 = train_encoder_data[current:nextone]
    train_X2 = train_decoder_data[current:nextone]
    portion = train_target_data[current:nextone]
    output = utils.to_categorical(portion, max_words)
#     print(train_X1.shape)
#     print(train_X2.shape)
#     print(output.shape)
#     print(output)
    model.fit([train_X1 , train_X2], output, batch_size=32, epochs=20 ) 

Currently--> 0:300
Epoch 1/20
300/300 [==============================] - ETA: 58s - loss: 9.90 - ETA: 42s - loss: 9.90 - ETA: 33s - loss: 9.89 - ETA: 27s - loss: 9.84 - ETA: 21s - loss: 9.67 - ETA: 16s - loss: 9.45 - ETA: 11s - loss: 9.21 - ETA: 6s - loss: 8.9857 - ETA: 1s - loss: 8.783 - 45s 150ms/step - loss: 8.7043
Epoch 2/20
300/300 [==============================] - ETA: 37s - loss: 6.77 - ETA: 32s - loss: 6.77 - ETA: 28s - loss: 6.77 - ETA: 24s - loss: 6.74 - ETA: 19s - loss: 6.69 - ETA: 15s - loss: 6.69 - ETA: 10s - loss: 6.71 - ETA: 6s - loss: 6.6975 - ETA: 1s - loss: 6.692 - 43s 142ms/step - loss: 6.6787
Epoch 3/20


KeyboardInterrupt: 

In [ ]:
model.save("chat_botV2Large.h5")